# Housing in Victoria - Data Wrangling

    
Environment: Python 3.8.5 and Anaconda 4.10.3 (64-bit)
    
Libraries used:
    
- geopandas 0.10.2 (for working with geospatial data (the provided shape files), included in Anaconda Python 3.8.5)
- pandas 1.1.3 (for creation and manipulation of dataframes, included in Anaconda Python 3.8.5)
- math (for making use of the methods provided to perform mathematical tasks, included in Anaconda Python 3.8.5)
- datetime (for manipulation of time, included in Anaconda Python 3.8.5)
- shapely.geometry 1.7.1 (for manipulation of planar features (Points, Polygons), included in Anaconda 3.8.5)
- requests 2.24.0 (for making HTTP requests via Python, included in Anaconda 3.8.5)
- bs4 (for pulling out data out of HTML and XML files, included in Anaconda 3.8.5)
- urllib.request (for fetching and opening URLs, included in Anaconda 3.8.5)
- json 2.0.9 (for working with JSON data, included in Anaconda 3.8.5)
- re 2.2.1 (for regular expressions, included in Anaconda 3.8.5)
- sklearn.preprocessing (for transforming features (scaling, for instance), included in Anaconda 3.8.5)
- matplotlib 3.3.2 (for data visualization, included in Anaconda 3.8.5)

## 1. Introduction
This assessment comprises of the following tasks:
1. Data integration - Integrating data and datasets from various sources into a single dataset that matches the required schema.
2. Detection and correction of any possible issues resulted due to the integration tasks performed in the above step.

The following sections give a detailed explanation of this process.

## 2. Importing Libraries

Importing all the required libraries to carry out the outlined tasks.

In [1]:
import geopandas as gpd
import pandas as pd
import math
from datetime import timedelta
from shapely.geometry import Point
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from pandas import json_normalize
import re
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Umasree\anaconda3\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


## 3. Task: Data Integration

The following sections give a detailed explanation of the steps carried out in this task.

### 3.1 Creation of Dataframe

#### 3.1.1 XML file

This step includes the parsing of the given XML file and then storing the retrived data in a dataframe.

First, using file methods, the file will be opened (using '`open()`' method), its contents will be read (using '`readlines()`' method) and stored in a list ('`xml_content`').

Since an XML file requires a single root element (and therefore is the parent element to all the other elements), we'll add that as it is missing in the given file.
The root and its corresponding closing tag will be: '`<properties>`' and '`</properties>`'
The root needs to be the first element, hence this will be inserted at index '0' and the closing tag will be appended to the end of the list.

Also, any '`&`' will be replaced with '`&amp;`'.

Once these necessary changes have been made, the contents will be written to a file, which will then be used later for parsing.

In [2]:
xml_input = open("31363660.xml", "r")

xml_content = xml_input.readlines()
xml_input.close()

xml_content

['<property>\n',
 '  <property_id>91099</property_id>\n',
 '  <lat>-37.95640182</lat>\n',
 '  <lng>145.0725403</lng>\n',
 '  <addr_street>99 Herald Street</addr_street>\n',
 '</property>\n',
 '<property>\n',
 '  <property_id>65182</property_id>\n',
 '  <lat>-37.80659</lat>\n',
 '  <lng>145.317902</lng>\n',
 '  <addr_street>18 Kilsyth Avenue</addr_street>\n',
 '</property>\n',
 '<property>\n',
 '  <property_id>17661</property_id>\n',
 '  <lat>-37.698942</lat>\n',
 '  <lng>144.909829</lng>\n',
 '  <addr_street>73 AUGUSTINE TERRACE</addr_street>\n',
 '</property>\n',
 '<property>\n',
 '  <property_id>46534</property_id>\n',
 '  <lat>-37.70618</lat>\n',
 '  <lng>145.139217</lng>\n',
 '  <addr_street>125 Ryans Road</addr_street>\n',
 '</property>\n',
 '<property>\n',
 '  <property_id>87356</property_id>\n',
 '  <lat>-37.939583</lat>\n',
 '  <lng>145.014479</lng>\n',
 '  <addr_street>133 Thomas Street</addr_street>\n',
 '</property>\n',
 '<property>\n',
 '  <property_id>59265</property_id>\n

In [3]:
xml_content.append("\n</properties>")
xml_content.insert(0, '<properties>\n')

In [4]:
#looping over the list and replacing the '&' with '&amp;' (if present)
for idx in range(len(xml_content)):
    if "&" in xml_content[idx]:
        xml_content[idx] = xml_content[idx].replace("&", "&amp;")

In [5]:
xml_output = open("31363660.xml", "w")

for line in xml_content:
    xml_output.write(line)
    
xml_output.close()

Using BeautifulSoup, the XML file contents will be extracted.

A BeautifulSoup object '`btree`' will be created.

In [6]:
btree = BeautifulSoup(open("31363660.xml"), "lxml-xml")  #BeautifulSoup object creation
print(btree.prettify())

<?xml version="1.0" encoding="utf-8"?>
<properties>
 <property>
  <property_id>
   91099
  </property_id>
  <lat>
   -37.95640182
  </lat>
  <lng>
   145.0725403
  </lng>
  <addr_street>
   99 Herald Street
  </addr_street>
 </property>
 <property>
  <property_id>
   65182
  </property_id>
  <lat>
   -37.80659
  </lat>
  <lng>
   145.317902
  </lng>
  <addr_street>
   18 Kilsyth Avenue
  </addr_street>
 </property>
 <property>
  <property_id>
   17661
  </property_id>
  <lat>
   -37.698942
  </lat>
  <lng>
   144.909829
  </lng>
  <addr_street>
   73 AUGUSTINE TERRACE
  </addr_street>
 </property>
 <property>
  <property_id>
   46534
  </property_id>
  <lat>
   -37.70618
  </lat>
  <lng>
   145.139217
  </lng>
  <addr_street>
   125 Ryans Road
  </addr_street>
 </property>
 <property>
  <property_id>
   87356
  </property_id>
  <lat>
   -37.939583
  </lat>
  <lng>
   145.014479
  </lng>
  <addr_street>
   133 Thomas Street
  </addr_street>
 </property>
 <property>
  <property_id>
   59

Using the '`find_all()`' function, information from each tag will be extracted. The tags include:

- property_id: stores the unique identifier of each property
- lat: latitude value of the property
- lng: longitude value of the property
- addr_street: address of the property

The information from each tag will be stored in the corresponding lists:
- property_id: list that stores all the ids
- latitudes: list for all the latitude values
- longitudes: list for all the longitude values
- address: list for all the addresses

In [7]:
property_id = [int(pid.string) for pid in btree.find_all("property_id")]
latitudes = [float(lat.string) for lat in btree.find_all("lat")]
longitudes = [float(long.string) for long in btree.find_all("lng")]
address = [addr.string for addr in btree.find_all("addr_street")]

After extraction, all the information is first stored in a dictionary ('`xml_dict`') and then this is used in creation of a Pandas DataFrame ('`xml_df`').

In [8]:
xml_dict = dict()

xml_dict['property_id'] = property_id
xml_dict['lat'] = latitudes
xml_dict['lng'] = longitudes
xml_dict['addr_street'] = address

xml_df = pd.DataFrame(xml_dict)
xml_df.head()

,property_id,lat,lng,addr_street
0,91099,-37.956402,145.072540,99 Herald Street
1,65182,-37.806590,145.317902,18 Kilsyth Avenue
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE
3,46534,-37.706180,145.139217,125 Ryans Road
4,87356,-37.939583,145.014479,133 Thomas Street


In [9]:
xml_df.shape  #dimensions

(1124, 4)

#### 3.1.2 JSON file

Here, we'll parse the given JSON file and then produce a dataframe that'll store the extracted information.

First, we'll open the file and use the '`load()`' method from the '`json`' module to store the content.

In [10]:
with open("31363660.json") as json_file:
    data = json.load(json_file)

In [11]:
data[0] #having a look at the first element of the extracted data

{'property_id': 69735,
 'lat': -37.870465,
 'lng': 145.071576,
 'addr_street': '20 St Georges Crescent'}

Using the '`json_normalize()`' function from the '`Pandas`' library, we'll store the extracted information in a dataframe ('`json_df`').

In [12]:
json_df = json_normalize(data)

In [13]:
json_df.head()

,property_id,lat,lng,addr_street
0,69735,-37.870465,145.071576,20 St Georges Crescent
1,13583,-37.697856,144.748974,9 Penshurst Court
2,20771,-37.762518,144.958547,20 Osborne Street
3,76248,-37.902363,145.096580,49 Burlington St
4,49402,-37.806021,145.092510,32 Beckett Street


In [14]:
json_df.shape

(1129, 4)

#### 3.1.3 Merging dataframes

Next, we'll merge both the dataframes ('`xml_df`' and '`json_df`') to get our final dataframe, where the '`pd.concat()`' function will be used.

We'll also address any possible issues due to this concatenation/merging (for instance, duplicate rows/records).

In [15]:
df = pd.concat([xml_df, json_df], ignore_index = True)
df.head()

,property_id,lat,lng,addr_street
0,91099,-37.956402,145.072540,99 Herald Street
1,65182,-37.806590,145.317902,18 Kilsyth Avenue
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE
3,46534,-37.706180,145.139217,125 Ryans Road
4,87356,-37.939583,145.014479,133 Thomas Street


In [16]:
df.shape

(2253, 4)

Verifying if there are any duplicates in the '`property_id`' column as they're supposed to be unique for each property.

In [17]:
len(df['property_id'].unique())  #checking for the number of unique ids

2225

Since the total number of rows are 2253 but there are only 2225 unique ids, there are duplicates. These duplicates will be dropped using the '`drop_duplicates()`' method.

In [18]:
dups = df[df.duplicated(['property_id'])] #having a look at the duplicates
dups

,property_id,lat,lng,addr_street
652,53782,-37.828386,145.012420,8 Loyola Grove
875,17463,-37.711500,144.904762,57 Tarana Avenue
936,77123,-37.943287,145.115295,16 Murphy Street
961,94436,-37.954779,145.084553,8 Muirfield Close
978,96635,-38.010936,145.279590,21 Glenburn Drive
997,54758,-37.815524,145.019123,9 Brook Street
1101,87897,-37.936626,145.021669,66 Teddington Road
1124,69735,-37.870465,145.071576,20 St Georges Crescent
1125,13583,-37.697856,144.748974,9 Penshurst Court
1126,20771,-37.762518,144.958547,20 Osborne Street


In [19]:
dups.shape 

(28, 4)

In [20]:
df = df.drop_duplicates(subset = ['property_id'])  #dropping duplicates based on the 'property_id' column

In [21]:
df.shape

(2225, 4)

### 3.2 'suburb' column

Next, we'll try to fill the '`suburb`' column using the given shapefile. We'll use a GeoPandas DataFrame to store the information of the shapefile.

In [22]:
#for ensuring the columns meet the required format
df['lat'] = df['lat'].round(6)
df['lng'] = df['lng'].round(6)

In [23]:
vic_data = gpd.read_file("vic_suburb_bounadry-20211012T112223Z-001/vic_suburb_bounadry/VIC_LOCALITY_POLYGON_shp.shp")  #reading the shapefile

In [24]:
vic_data.head()

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."
2,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"POLYGON ((142.92288 -37.97886, 142.90449 -37.9..."
3,6673,2011-08-31,None,VIC734,2017-08-09,None,DEPTFORD,None,None,G,None,2,"POLYGON ((147.82336 -37.66001, 147.82313 -37.6..."
4,6674,2011-08-31,None,VIC2900,2012-04-27,None,YANAC,None,None,G,None,2,"POLYGON ((141.27978 -35.99859, 141.27989 -35.9..."


In [25]:
vic_data.shape

(2973, 13)

In [26]:
vic_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   LC_PLY_PID  2973 non-null   object  
 1   DT_CREATE   2973 non-null   object  
 2   DT_RETIRE   0 non-null      object  
 3   LOC_PID     2973 non-null   object  
 4   VIC_LOCALI  2973 non-null   object  
 5   VIC_LOCA_1  0 non-null      object  
 6   VIC_LOCA_2  2973 non-null   object  
 7   VIC_LOCA_3  0 non-null      object  
 8   VIC_LOCA_4  31 non-null     object  
 9   VIC_LOCA_5  2973 non-null   object  
 10  VIC_LOCA_6  0 non-null      object  
 11  VIC_LOCA_7  2973 non-null   object  
 12  geometry    2973 non-null   geometry
dtypes: geometry(1), object(12)
memory usage: 302.1+ KB


The dataframe '`vic_data`' consists of a column named '`geometry`' that holds planar features information for each record.
Also, looks like the '`VIC_LOCA_2`' holds the names of all the suburbs.

In [27]:
df['suburb'] = 'Not available'  #addition of the 'suburb' column, with the default value 'Not available'
df

,property_id,lat,lng,addr_street,suburb
0,91099,-37.956402,145.072540,99 Herald Street,Not available
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,Not available
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,Not available
3,46534,-37.706180,145.139217,125 Ryans Road,Not available
4,87356,-37.939583,145.014479,133 Thomas Street,Not available
...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,Not available
2248,56038,-37.822793,145.066628,95 Broadway,Not available
2249,63480,-37.783340,145.287864,13 Aminga Court,Not available
2251,80587,-37.989635,145.171660,9 Wannon Court,Not available


We'll use the coordinates (latitude, longitude) of each property to check in which suburb does it actually lie.

First, for each row, we'll extract the coordinates and then convert it into a '`Point`' object (from '`shapely.geometry`') and check in which '`Polygon`' object does it lie (here, each entry in the '`geometry`' column is a '`Polygon`' object) using the '`within`' method and then assign the suburb.

In [28]:
for idx in list(df.index):
    coord = Point(df['lng'][idx], df['lat'][idx])  #Point conversion
    for ix in list(vic_data.index):
        if coord.within(vic_data['geometry'][ix]): 
            df.at[idx, 'suburb'] = vic_data['VIC_LOCA_2'][ix]

In [29]:
df

,property_id,lat,lng,addr_street,suburb
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON
...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH


### 3.3 Local Government Area ('lga' column)

In this section, based on the suburb of the property, the LGA will be assigned.

#### 3.3.1 Parsing PDF file

First, we need to parse the PDF file that stores the LGA and its corresponding suburb values.

In [30]:
#!pdf2txt.py -o lga_to_suburb.txt lga_to_suburb.pdf  #parsing using pdfminer

After parsing, we'll have a look at the text file to see what it looks like so that we can extract all the required content and use it later for assigning the LGA value.

Using file methods, we'll open and read the content (content is stored in the list '`lga_list`'). We'll then perform any preprocessing, if required, to finally obtain our required content.

In [31]:
file = open("lga_to_suburb.txt", 'r')

lga_list = file.readlines()
file.close()

In [32]:
lga_list  #having a look at the list

["BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'] \n",
 '\n',
 "BRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'] \n",
 '\n',
 "DAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava'] \n",
 '\n',
 "HOBSONS BAY : ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East'] \n",
 '\n',
 "HUME : ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East'] \n",
 '\n',
 "MARIBYRNONG : ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood'] \n",
 '\n',
 "MELBOURNE : ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie'] \n",
 '\n',
 "MELTON : ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone'] \n",
 '\n',
 "MOONEE VALLEY : ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood'] \n",
 '\n',
 "MORELAND : ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood'] \n",
 '\n',
 "NILLUMBIK : ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy', 'Plenty'] \n",
 '\n',
 "WHITTLESEA : ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris'] \n",
 '\n',

In [33]:
lga_list = lga_list[:-1]  #the last element seems to be some string that isn't relevant, hence its excluded.

In [34]:
lga_list = [lga_list[idx] for idx in range(len(lga_list)) if idx % 2 == 0]  #excluding all the '\n' strings at odd index positions

In [35]:
lga_list  #let's look at the list now

["BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'] \n",
 "BRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'] \n",
 "DAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava'] \n",
 "HOBSONS BAY : ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East'] \n",
 "HUME : ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East'] \n",
 "MARIBYRNONG : ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood'] \n",
 "MELBOURNE : ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie'] \n",
 "MELTON : ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone'] \n",
 "MOONEE VALLEY : ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood'] \n",
 "MORELAND : ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood'] \n",
 "NILLUMBIK : ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy', 'Plenty'] \n",
 "WHITTLESEA : ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris'] \n",
 "WYNDHAM : ['Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton'] \n",
 "YARRA : ['Hampto

In [36]:
for idx in range(len(lga_list)):  #iterating over the 'lga_list' list
    lga_list[idx] = lga_list[idx].replace("\n", "")  #replacing the newline character with an empty string 
    lga_list[idx] = lga_list[idx].split(":")  #splitting the element/string

In [37]:
lga_list  #checking what the list looks like now

[['BANYULE ', " ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'] "],
 ['BRIMBANK ', " ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'] "],
 ['DAREBIN ', " ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava'] "],
 ['HOBSONS BAY ',
  " ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East'] "],
 ['HUME ', " ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East'] "],
 ['MARIBYRNONG ', " ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood'] "],
 ['MELBOURNE ', " ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie'] "],
 ['MELTON ',
  " ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone'] "],
 ['MOONEE VALLEY ',
  " ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood'] "],
 ['MORELAND ', " ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood'] "],
 ['NILLUMBIK ',
  " ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy', 'Plenty'] "],
 ['WHITTLESEA ',
  " ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris'] "],
 ['WYNDHAM ', " ['Glenroy', 'Gowanbra

Now each element in the list is list that contains two elements: the LGA as the first element and the string of suburbs for the corresponding LGA as the second element.
We'll try preprocessing these in the following steps:

In [39]:
#preprocessing the second element of each sublist (suburbs)
for idx in range(len(lga_list)):
    lga_list[idx][1] = lga_list[idx][1].replace("[", "")  #replacing the character '[' with an empty string
    lga_list[idx][1] = lga_list[idx][1].replace("]", "")  #replacing the character ']' with an empty string
    lga_list[idx][1] = lga_list[idx][1].replace("'", "")  #replacing the character "'" with an empty string
    lga_list[idx][1] = lga_list[idx][1].split(",")  #splitting the string
    
    lga_list[idx][0] = lga_list[idx][0].strip()  #stripping of any whitespace characters in the first element (lGA)
    
    for i in range(len(lga_list[idx][1])):
        lga_list[idx][1][i] = lga_list[idx][1][i].strip()  #stripping any whitespace characters from each suburb

In [40]:
lga_list

[['BANYULE', ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']],
 ['BRIMBANK', ['Altona', 'Altona North', 'Armadale', 'Ascot Vale']],
 ['DAREBIN', ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava']],
 ['HOBSONS BAY', ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East']],
 ['HUME', ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East']],
 ['MARIBYRNONG', ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood']],
 ['MELBOURNE', ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie']],
 ['MELTON', ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone']],
 ['MOONEE VALLEY', ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood']],
 ['MORELAND', ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood']],
 ['NILLUMBIK',
  ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy', 'Plenty']],
 ['WHITTLESEA', ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris']],
 ['WYNDHAM', ['Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton']],
 ['YARRA',
  ['Hampton East', 'Hawthorn', 

Since it looks good now, we'll store this extracted information in a dictionary '`lga_dict`', where the key is the LGA and the value will be the corresponding list of suburbs.
Using this dictionary we'll assign the LGA for each row.

In [41]:
lga_dict = dict()

for idx in range(len(lga_list)):
    lga_dict[lga_list[idx][0]] = lga_list[idx][1]

In [42]:
lga_dict

{'BANYULE': ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'],
 'BRIMBANK': ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'],
 'DAREBIN': ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava'],
 'HOBSONS BAY': ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East'],
 'HUME': ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East'],
 'MARIBYRNONG': ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood'],
 'MELBOURNE': ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie'],
 'MELTON': ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone'],
 'MOONEE VALLEY': ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood'],
 'MORELAND': ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood'],
 'NILLUMBIK': ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy', 'Plenty'],
 'WHITTLESEA': ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris'],
 'WYNDHAM': ['Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton'],
 'YARRA': ['Hampton East',
  'Hawthorn',
  'Heidelberg Heights',
  'He

#### 3.3.2 'lga' column

Using the dictionary from above, we'll assign the LGA for each row.

In [43]:
df['lga'] = 'Not available'  #addition of new column, 'lga', with default value 'Not available'

In [44]:
for idx in list(df.index):
    suburb = df['suburb'][idx]  #extract the suburb value for the row
    for lga, suburbs in lga_dict.items():
        suburbs = [suburb.upper() for suburb in suburbs]  #upper method for conversion of each suburb to uppercase suburb value for matching  
        if suburb.upper() in suburbs:  #checking if the suburb exists in the list 'suburbs'
            df.at[idx, 'lga'] = lga  #if yes, corresponding LGA is assigned

In [45]:
df

,property_id,lat,lng,addr_street,suburb,lga
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM
...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON


### 3.4 GTFS Data

First, we'll have a look at what the files are:

In [46]:
agency = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/agency.txt")
agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,1,PTV,http://www.ptv.vic.gov.au,Australia/Melbourne,EN


In [47]:
calendar = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/calendar.txt")
calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,T2,0,0,0,0,0,1,0,20151009,20151011
1,UJ,0,0,0,0,0,0,1,20151009,20151011
2,T6,0,0,0,0,1,0,0,20151009,20151011
3,T5,1,1,1,1,0,0,0,20151012,20151015
4,T2_1,0,0,0,0,0,1,0,20151016,20151018


In [48]:
calendar.shape

(19, 10)

In [49]:
calendar_dates = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/calendar_dates.txt")
calendar_dates

,service_id,date,exception_type
0,T0,20151103,2
1,T0+a5,20151103,2


In [50]:
routes = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/routes.txt")
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,2-ALM-B-mjp-1,1,Alamein,Alamein - City (Flinders Street),2
1,2-ALM-C-mjp-1,1,Alamein,Alamein - City (Flinders Street),2
2,2-ALM-D-mjp-1,1,Alamein,Alamein - City (Flinders Street),2
3,2-ALM-E-mjp-1,1,Alamein,Alamein - City (Flinders Street),2
4,2-ALM-F-mjp-1,1,Alamein,Alamein - City (Flinders Street),2


In [51]:
routes.shape

(81, 5)

In [52]:
shapes = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/shapes.txt")
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,2-ain-mjp-1.1.H,-37.818631,144.951994,1,0.000000
1,2-ain-mjp-1.1.H,-37.817425,144.951050,2,157.543645
2,2-ain-mjp-1.1.H,-37.817241,144.950828,3,185.827916
3,2-ain-mjp-1.1.H,-37.816327,144.950047,4,308.469671
4,2-ain-mjp-1.1.H,-37.816127,144.949950,5,332.239399


In [53]:
shapes.shape

(339711, 5)

In [54]:
stop_times = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/stop_times.txt")
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,17182517.T2.2-ALM-B-mjp-1.1.H,04:57:00,04:57:00,19847,1,NaN,0,0,0.000000
1,17182517.T2.2-ALM-B-mjp-1.1.H,04:58:00,04:58:00,19848,2,NaN,0,0,723.017818
2,17182517.T2.2-ALM-B-mjp-1.1.H,05:00:00,05:00:00,19849,3,NaN,0,0,1951.735072
3,17182517.T2.2-ALM-B-mjp-1.1.H,05:02:00,05:02:00,19850,4,NaN,0,0,2899.073349
4,17182517.T2.2-ALM-B-mjp-1.1.H,05:04:00,05:04:00,19851,5,NaN,0,0,3927.090952


In [55]:
stop_times.shape

(390305, 9)

In [56]:
stops = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/stops.txt")
stops.head()

,stop_id,stop_name,stop_short_name,stop_lat,stop_lon
0,15351,Sunbury Railway Station,Sunbury,-37.579091,144.727319
1,15353,Diggers Rest Railway Station,Diggers Rest,-37.627017,144.719922
2,19827,Stony Point Railway Station,Crib Point,-38.374235,145.221837
3,19828,Crib Point Railway Station,Crib Point,-38.366123,145.204043
4,19829,Morradoo Railway Station,Crib Point,-38.354033,145.189602


In [57]:
stops.shape

(218, 5)

In [58]:
trips = pd.read_csv("Vic_GTFS_data -20211012T112155Z-001/Vic_GTFS_data/metropolitan/trips.txt")
trips.head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
1,2-ALM-F-mjp-1,T0,17067988.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
2,2-ALM-F-mjp-1,T0,17067992.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
3,2-ALM-F-mjp-1,T0,17067999.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
4,2-ALM-F-mjp-1,T0,17068003.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0


In [59]:
trips.shape

(23809, 6)

#### 3.4.1 'closest_train_station_id' and 'distance_to_closest_train_station' columns

The '`stops`' dataframe holds information of all train stations and hence will be used to calculate for each property the closest train station and the distance (based on Haversine formula).

In [60]:
#addition of two new columns with their default values
df['closest_train_station_id'] = 0
df['distance_to_closest_train_station'] = 0.0

In [61]:
def closestStation(source, destination):
    
    '''
    This function will return the closest station (destination) for a given property (source) based on the
    Haversine distance.
    The coordinates of both the source and destination will be extracted and converted to radians.
    The distance (in km) is calculated using the Haversine formula.
    '''
    
    x1, y1 = source[0], source[1]  #source/property latitude and longitude
    x2, y2 = destination[0], destination[1]  #destination/train station latitude and longitude
    #converting lats and long to radians, as sine and cosine functions produce proper output here in terms of radians
    x1 = ((x1) * math.pi) / 180
    x2 = ((x2) * math.pi) / 180
    y1 = ((y1) * math.pi) / 180
    y2 = ((y2) * math.pi) / 180
    radius = 6378
    result = (((math.sin((x2 - x1)/2)) ** 2) + ((math.cos(x1) * math.cos(x2)) * ((math.sin((y2 - y1)/2)) ** 2)))
    distance = 2 * radius * math.asin(math.sqrt(result))
    return distance

The coordinates for each train station will be stored in a dictionary named '`destination_dict`', that'll be used for calculating the distance between each property and these train stations to pick the closest one based on the distance that'll be calculated using the above defined function.

In [62]:
destination_dict = dict()

stops_list = stops['stop_id'].tolist()  #storing all train station ids in a list

for idx in list(stops.index):  
    coords = (stops['stop_lat'][idx], stops['stop_lon'][idx])  #extraction of coordinates of each station
    destination_dict[stops_list[idx]] = coords  #storing the coordinates as value in the dictionary with each station as key

In [63]:
destination_dict

{15351: (-37.579090970578704, 144.727318781329),
 15353: (-37.627016514935704, 144.719922441113),
 19827: (-38.3742345364937, 145.221837462187),
 19828: (-38.3661233827862, 145.204043321601),
 19829: (-38.35403333658439, 145.189602487712),
 19830: (-38.3373903202406, 145.178026527511),
 19831: (-38.3056585273722, 145.185979882012),
 19832: (-38.2598150046916, 145.186400708984),
 19833: (-38.2253419593114, 145.17624491889998),
 19834: (-38.1940430573051, 145.16052608761498),
 19835: (-37.8593084165858, 145.058224898221),
 19836: (-38.1520339975222, 145.139533963382),
 19837: (-37.8689568446092, 145.06295082624501),
 19838: (-37.8769300156728, 145.0693930023),
 19839: (-37.874402220049795, 145.090662027436),
 19840: (-37.873603393318795, 145.112090156436),
 19841: (-37.811981307380705, 144.955653760429),
 19842: (-37.809938766738604, 144.962593535096),
 19843: (-37.8110540555305, 144.972910916416),
 19844: (-37.909102097471205, 145.35529128285802),
 19845: (-37.9081184864021, 145.3429945

For each property, the distance between it and every train station in the '`destination_dict`' will be calculated by calling the '`closestStation`' function and stored in the dictionary '`distance_dict`'.
The minimum value/distance will be obtained using '`min()`' and the corresponding train station id will be extracted from the '`distance_dict`' dictionary. These will then be assigned, for that property, in the appropriate columns.

In [64]:
for idx in list(df.index):
    source = (df['lat'][idx], df['lng'][idx])  #extracting coordinates of the property
    
    distance_dict = dict()  
    
    #calculation of distance from property to each station
    for stop in destination_dict.keys(): 
        distance = closestStation(source, destination_dict[stop])
        distance_dict[stop] =  distance
        
    min_dist = min(distance_dict.values())
    stop_id = [stop for stop in distance_dict.keys() if distance_dict[stop] == min_dist]
    df.at[idx, 'closest_train_station_id'] = stop_id[0]  #assigns closest train station id
    df.at[idx, 'distance_to_closest_train_station'] = round(min_dist, 6)  #assigns the corresponding distance

In [65]:
df

,property_id,lat,lng,addr_street,suburb,lga,closest_train_station_id,distance_to_closest_train_station
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES,19866,1.947320
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST,19877,2.478957
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM,20031,0.679184
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT,19987,1.116812
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM,19949,1.155951
...,...,...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP,19900,1.181700
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE,19892,0.371542
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA,19878,1.490612
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON,19889,2.160029


#### 3.4.2 'travel_min_to_MC' and 'direct_journey_flag' columns

Since we've calculated the closest train station id for each property, we'll use it to check if there are any direct trips to the '`Melbourne Central Railway Station`'. If there are any ('`1`' will be added to the '`direct_journey_flag`' column), the rounded travel time will be added.

In [66]:
#addition of new columns with corresponding default values
df['travel_min_to_MC'] = "-1"
df['direct_journey_flag'] = -1

First, we'll find the '`stop_id`' of the '`Melbourne Central Railway Station`'.

In [67]:
stops[stops.stop_name == "Melbourne Central Railway Station"]

,stop_id,stop_name,stop_short_name,stop_lat,stop_lon
17,19842,Melbourne Central Railway Station,Melbourne City,-37.809939,144.962594


The '`calendars`' dataframe holds information regarding on what day the service (identified by '`service_id`') operates. Hence, we'll extract all those service ids that run on all weekdays only and store it in a list named '`service_weekdays`'.

In [68]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,T2,0,0,0,0,0,1,0,20151009,20151011
1,UJ,0,0,0,0,0,0,1,20151009,20151011
2,T6,0,0,0,0,1,0,0,20151009,20151011
3,T5,1,1,1,1,0,0,0,20151012,20151015
4,T2_1,0,0,0,0,0,1,0,20151016,20151018
5,UJ_1,0,0,0,0,0,0,1,20151016,20151018
6,T6_1,0,0,0,0,1,0,0,20151016,20151018
7,T5_1,1,1,1,1,0,0,0,20151019,20151022
8,T0,1,1,1,1,1,0,0,20151023,20151122
9,T2_2,0,0,0,0,0,1,0,20151023,20151122


In [69]:
service_weekdays = calendar[(calendar.monday == 1) & (calendar.tuesday == 1) & (calendar.wednesday == 1) & (calendar.thursday == 1) & (calendar.friday == 1)]['service_id'].tolist()
service_weekdays

['T0']

The '`trips`' dataframe includes information about trips (on what route it runs, the service day and the path it takes). Using the list '`service_weekdays`', we'll only keep those rows from the dataframe whose '`service_id`' exists in the list.

First, we'll create a copy dataframe of '`trips`', named '`trips_copy`', and perform the above mentioned steps.

In [70]:
trips_copy = trips.copy()
trips_copy

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
1,2-ALM-F-mjp-1,T0,17067988.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
2,2-ALM-F-mjp-1,T0,17067992.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
3,2-ALM-F-mjp-1,T0,17067999.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
4,2-ALM-F-mjp-1,T0,17068003.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0
...,...,...,...,...,...,...
23804,2-WMN-F-mjp-1,UJ_2,17072252.UJ.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1
23805,2-WMN-F-mjp-1,UJ_2,17072256.UJ.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1
23806,2-WMN-F-mjp-1,UJ_2,17072260.UJ.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1
23807,2-WMN-F-mjp-1,UJ_2,17072264.UJ.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1


In [71]:
#dropping all those rows whose 'service_id' isn't a part of the 'service_weekdays' list
for idx in list(trips_copy.index):
    if trips_copy['service_id'][idx] not in service_weekdays:
        trips_copy.drop(idx, inplace = True)

We'll now merge this dataframe with the '`routes`' dataframe on the common column '`route_id`'. Later we'll merge this obtained dataframe with the '`stop_times`' dataframe to obtain '`merged_two`' dataframe.
The '`stop_times`' contains additional information of each trip, including the stop it serves and its sequence, along with the arrival and departure times, whereas, '`routes`' holds descriptive information about all the routes in the GTFS feed.

The '`merged_two`' dataframe can help us retrieve information regarding the direct trips/journeys along with the travel time to the '`Melbourne Central Railway Station`'.

In [72]:
merged_one = pd.merge(trips_copy, routes, on = 'route_id')
merged_one

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id,agency_id,route_short_name,route_long_name,route_type
0,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2
1,2-ALM-F-mjp-1,T0,17067988.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2
2,2-ALM-F-mjp-1,T0,17067992.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2
3,2-ALM-F-mjp-1,T0,17067999.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2
4,2-ALM-F-mjp-1,T0,17068003.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2
...,...,...,...,...,...,...,...,...,...,...
2318,2-WMN-F-mjp-1,T0,17072078.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2
2319,2-WMN-F-mjp-1,T0,17072082.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2
2320,2-WMN-F-mjp-1,T0,17072087.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2
2321,2-WMN-F-mjp-1,T0,17072091.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2


In [73]:
merged_two = pd.merge(merged_one, stop_times, on = "trip_id")
merged_two

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id,agency_id,route_short_name,route_long_name,route_type,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2,05:01:00,05:01:00,19847,1,NaN,0,0,0.000000
1,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2,05:02:00,05:02:00,19848,2,NaN,0,0,723.017818
2,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2,05:04:00,05:04:00,19849,3,NaN,0,0,1951.735072
3,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2,05:06:00,05:06:00,19850,4,NaN,0,0,2899.073349
4,2-ALM-F-mjp-1,T0,17067982.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,City (Flinders Street),0,1,Alamein,Alamein - City (Flinders Street),2,05:08:00,05:08:00,19851,5,NaN,0,0,3927.090952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42895,2-WMN-F-mjp-1,T0,17072091.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2,23:59:00,23:59:00,19991,4,NaN,0,0,3641.811422
42896,2-WMN-F-mjp-1,T0,17072097.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2,24:33:00,24:33:00,19994,1,NaN,0,0,0.000000
42897,2-WMN-F-mjp-1,T0,17072097.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2,24:35:00,24:35:00,19993,2,NaN,0,0,1702.554760
42898,2-WMN-F-mjp-1,T0,17072097.T0.2-WMN-F-mjp-1.6.R,2-WMN-F-mjp-1.6.R,Williamstown,1,1,Williamstown,Williamstown - City (Flinders Street),2,24:37:00,24:37:00,19992,3,NaN,0,0,2598.738912


Since we're only interested in direct journeys departing between 7-9 AM, we'll filter the '`merged_two`' dataframe to exclude records/rows that do not match this condition. Also, we'll drop some columns that are not relevant for our analysis.

In [74]:
merged_two = merged_two[((merged_two.departure_time >= "07:00:00") & (merged_two.departure_time <= "09:00:00"))]

In [75]:
merged_two.drop(['stop_headsign', 'pickup_type', 'drop_off_type', 'agency_id', 'route_short_name', 'route_type', 'shape_dist_traveled'], axis = 1, inplace = True)

C:\Users\Umasree\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [76]:
merged_two

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id,route_long_name,arrival_time,departure_time,stop_id,stop_sequence
310,2-ALM-F-mjp-1,T0,17068385.T0.2-ALM-F-mjp-1.2.H,2-ALM-F-mjp-1.2.H,City (Flinders Street),0,Alamein - City (Flinders Street),07:00:00,07:00:00,19904,9
311,2-ALM-F-mjp-1,T0,17068385.T0.2-ALM-F-mjp-1.2.H,2-ALM-F-mjp-1.2.H,City (Flinders Street),0,Alamein - City (Flinders Street),07:02:00,07:02:00,19905,10
312,2-ALM-F-mjp-1,T0,17068385.T0.2-ALM-F-mjp-1.2.H,2-ALM-F-mjp-1.2.H,City (Flinders Street),0,Alamein - City (Flinders Street),07:04:00,07:04:00,19906,11
313,2-ALM-F-mjp-1,T0,17068385.T0.2-ALM-F-mjp-1.2.H,2-ALM-F-mjp-1.2.H,City (Flinders Street),0,Alamein - City (Flinders Street),07:07:00,07:07:00,19908,13
314,2-ALM-F-mjp-1,T0,17068385.T0.2-ALM-F-mjp-1.2.H,2-ALM-F-mjp-1.2.H,City (Flinders Street),0,Alamein - City (Flinders Street),07:10:00,07:10:00,19843,14
...,...,...,...,...,...,...,...,...,...,...,...
42795,2-WMN-F-mjp-1,T0,17071157.T0.2-WMN-F-mjp-1.4.R,2-WMN-F-mjp-1.4.R,Williamstown,1,Williamstown - City (Flinders Street),08:57:00,08:57:00,19992,11
42797,2-WMN-F-mjp-1,T0,17071159.T0.2-WMN-F-mjp-1.4.R,2-WMN-F-mjp-1.4.R,Williamstown,1,Williamstown - City (Flinders Street),08:51:00,08:51:00,19854,1
42798,2-WMN-F-mjp-1,T0,17071159.T0.2-WMN-F-mjp-1.4.R,2-WMN-F-mjp-1.4.R,Williamstown,1,Williamstown - City (Flinders Street),08:54:00,08:54:00,22180,2
42799,2-WMN-F-mjp-1,T0,17071159.T0.2-WMN-F-mjp-1.4.R,2-WMN-F-mjp-1.4.R,Williamstown,1,Williamstown - City (Flinders Street),08:57:00,08:57:00,19973,3


The below defined function will help in the calculation of the travel time. The function does the following:

For each property's closest station, the trip ids that include the station id are stored in the '`trips_ids`' list, where each trip id will be used to generate/produce a subset from the '`merged_two`' dataframe ('`subset`'). From the subset, we'll look if the '`stop_id`' of the '`Melbourne Central Railway Station`' is present or not. If it is, we can proceed to the next step, otherwise, it'll imply that this trip is not direct. The trip id in the latter case will be appended to the list '`no_trip`'.
Next, the '`stop_sequence`' value for both the '`Melbourne Central Railway Station`' (if present) and the closest station id are recorded. If the '`Melbourne Central Railway Station`' sequence value is greater than that of the closest station's, this trip is valid and direct and this trip id will be appended to the '`valid_trip`' list.
After going through all the trip ids from the '`trips_ids`' list, the length of the '`valid_trip`' list is checked. If the length is 0, there is no travel time to calculate (because no direct trips). Otherwise, for each subset (obtained by the trip ids from the '`valid_trip`' list) of the '`merged_two`' dataframe ('`df`'), the time difference (in minutes) between the arrival time at the '`Melbourne Central Railway Station`' and the departure time from the closest train station id is calculated and appended to the'`travel_time`' list. The rounded travel time is then calculated and returned.

In [77]:
def getTravelTime(station_id):
    
    '''
    This function returns the rounded travel time to MC, if direct trips/journeys exist from the
    property's closest train station, otherwise it returns 'Not available'.
    
    '''
    
    trip_ids = list()
    
    #extraction of all trip ids that contain the closest station id
    for idx in list(merged_two.index):  
        if station_id == merged_two['stop_id'][idx]:
            trip_ids.append(merged_two['trip_id'][idx])

        no_trip = list()
        valid_trip = list()
    
    for tid in trip_ids:
        subset = merged_two[merged_two.trip_id == tid]
        row_ix = subset.index[subset.stop_id == station_id].tolist()[0] #extraction of row index of closest train station
        origin_seq_no = subset.loc[row_ix]['stop_sequence']  #stop sequence value of closest train station

        for ix in list(subset.index):
            if subset['stop_id'][ix] == 19842:
                dest_seq_no = subset['stop_sequence'][ix] #stop sequence of Melbourne Central

                if origin_seq_no < dest_seq_no:  #comparison of stop sequence values
                    valid_trip.append(tid)
                else:
                    pass
            else:
                no_trip.append(tid)

    travel_time = list()
    
    if len(valid_trip) > 0:
            
        for ix in range(len(valid_trip)):
            df = merged_two[merged_two.trip_id == valid_trip[ix]]

            org_ix = df.index[df.stop_id == station_id].tolist()[0]  #extraction of row index of closest train station
            dest_ix = df.index[df.stop_id == 19842].tolist()[0]  #extraction of row index of MC

            org_dept_time = df.loc[org_ix]['departure_time']  #dept time from closest station
            dest_arr_time = df.loc[dest_ix]['arrival_time']  #arrival time at MC
            
            #time objects for time difference calculation
            t1 = timedelta(hours = int(org_dept_time[:2]), minutes = int(org_dept_time[3:5]), seconds = int(org_dept_time[6:8]))
            t2 = timedelta(hours = int(dest_arr_time[:2]), minutes = int(dest_arr_time[3:5]), seconds = int(dest_arr_time[6:8]))
            diff = t2 - t1  #time difference
            diff_seconds = diff.seconds  #obtained answer is in seconds
            diff_minutes = diff_seconds / 60  #hence, is converted to minutes
            travel_time.append(diff_minutes)
        return str(float(round(sum(travel_time) / len(travel_time))))  #returns rounded travel time in minutes
            
    else:
        return "Not available"

The above function calculates the travel time only for those properties whose closest station isn't the MC. Otherwise, the value will be 0.

In [78]:
for idx in list(df.index):
    sid = df['closest_train_station_id'][idx]
    if sid != 19842:  #if station id not equal to MC's station id
        df.at[idx, 'travel_min_to_MC'] = getTravelTime(sid)
    else:
        df.at[idx, 'travel_min_to_MC'] = "0"

In [79]:
df

,property_id,lat,lng,addr_street,suburb,lga,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES,19866,1.947320,40.0,-1
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST,19877,2.478957,46.0,-1
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM,20031,0.679184,30.0,-1
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT,19987,1.116812,51.0,-1
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM,19949,1.155951,Not available,-1
...,...,...,...,...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP,19900,1.181700,31.0,-1
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE,19892,0.371542,18.0,-1
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA,19878,1.490612,42.0,-1
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON,19889,2.160029,43.0,-1


Now, if the travel time value is not '`Not available`', the value '`1`' is assigned to the '`direct_journey_flag`' column (including properties whose closest station is the MC). Otherwise, the value will be '`0`'. 

In [80]:
for idx in list(df.index):
    if df['travel_min_to_MC'][idx] == "Not available":
        df.at[idx, 'direct_journey_flag'] = 0
        
    elif df['closest_train_station_id'][idx] == 19842:
        df.at[idx, 'direct_journey_flag'] = 1
        
    else:
        df.at[idx, 'direct_journey_flag'] = 1

In [81]:
df

,property_id,lat,lng,addr_street,suburb,lga,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES,19866,1.947320,40.0,1
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST,19877,2.478957,46.0,1
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM,20031,0.679184,30.0,1
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT,19987,1.116812,51.0,1
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM,19949,1.155951,Not available,0
...,...,...,...,...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP,19900,1.181700,31.0,1
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE,19892,0.371542,18.0,1
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA,19878,1.490612,42.0,1
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON,19889,2.160029,43.0,1


### 3.5 Covid-19 positive cases

We'll first scrape the given website to extract the required information and store it. BeautifulSoup will be used for doing so, along with the methods/functions offered by the requests and urllib modules.

The information from the column '`CASES`' (in the website) will be extracted, along with the corresponding date from '`DATE`' for each LGA.

The extracted information will be stored in a dictionary.

In [82]:
#addition of new columns with their respective default values
df['30_sep_cases'] = 'Not available'
df['last_14_days_cases'] = 'Not available'
df['last_30_days_cases'] = 'Not available'
df['last_60_days_cases'] = 'Not available'

In [83]:
df

,property_id,lat,lng,addr_street,suburb,lga,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag,30_sep_cases,last_14_days_cases,last_30_days_cases,last_60_days_cases
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES,19866,1.947320,40.0,1,Not available,Not available,Not available,Not available
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST,19877,2.478957,46.0,1,Not available,Not available,Not available,Not available
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM,20031,0.679184,30.0,1,Not available,Not available,Not available,Not available
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT,19987,1.116812,51.0,1,Not available,Not available,Not available,Not available
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM,19949,1.155951,Not available,0,Not available,Not available,Not available,Not available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP,19900,1.181700,31.0,1,Not available,Not available,Not available,Not available
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE,19892,0.371542,18.0,1,Not available,Not available,Not available,Not available
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA,19878,1.490612,42.0,1,Not available,Not available,Not available,Not available
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON,19889,2.160029,43.0,1,Not available,Not available,Not available,Not available


#### 3.5.1 Scraping and storing

In each step, we'll try to extract the information stored in the '`href`' tags as these are links that will point to the next required page.

First, we'll send a HTTP request via '`requests.get()`' and check for the response status (using '`status_code`') to confirm that the response has been sent or not.

In [84]:
response = requests.get("https://covidlive.com.au/")

In [85]:
response.status_code  #value is 200, which means everything is fine

200

Using the response received, we'll start extracting the required links that'll point to next required page. Also, the '`re.compile()`' method will be used to search for those links that match the given/specified pattern within the method.

In [86]:
src = response.content

In [87]:
soupObj = BeautifulSoup(src, 'html.parser')  
links = soupObj.find("div", {"class":"box box1"}).findAll("a", href = re.compile("/vic"))  #the div tag and its class id have been obtained by inspecting the website
links

[<a href="/vic">Victoria</a>]

In [88]:
def getLink(links, phrase):
    
    '''
    This function returns the link that includes the specified phrase/text (given as input)
    from a list of links passed as input.
    '''
    
    for link in links:
        if phrase in link.text:
            return link.attrs['href']

In [89]:
result = getLink(links, "Victoria")  #calls the function with list 'links' and phrase 'Victoria' as input
result

'/vic'

In [90]:
src_2 = urlopen("https://covidlive.com.au/" + result) #from the result obtained above, open the specified url

In [91]:
soupObj = BeautifulSoup(src_2, 'html.parser')
links = soupObj.find("div", {"id":"menu-state"}).findAll("a", href =  re.compile("/vic/lga"))  #the div tag and its class id have been obtained by inspection

In [92]:
result = getLink(links, "LGA")  #calls the function with list 'links' and phrase 'LGA' as input
result

'/vic/lga'

In [93]:
src_3 = urlopen("https://covidlive.com.au/" + result)  #from the result obtained above, open the specified url

In [94]:
soupObj = BeautifulSoup(src_3, 'html.parser')
links = soupObj.find("section", {"CASES-BY-LGA MAP-3"}).find("h2").findAll("a")  #section tag and its corresponding class id retrieved by inspection

In [95]:
links

[<a href="/report/cases-by-lga/vic">VIC Cases LGA</a>]

In [96]:
result = getLink(links, "VIC Cases LGA")  #calls the function with list 'links' and phrase 'VIC Cases LGA' as input
result

'/report/cases-by-lga/vic'

In [97]:
src_4 =  urlopen("https://covidlive.com.au/" + result)  #from the result obtained above, open the specified url

In [98]:
soupObj = BeautifulSoup(src_4, 'html.parser')
links = soupObj.find("table", {"CASES-BY-LGA"}).findAll("td", {"class":"COL1 LGA"})  #obtained on inspection, to scrape table data

In [99]:
links

[<td class="COL1 LGA"><a class="metro" href="/vic/casey">Casey</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/greater-dandenong">Greater Dandenong</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/wyndham">Wyndham</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/melton">Melton</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/hume">Hume</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/whittlesea">Whittlesea</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/brimbank">Brimbank</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/moreland">Moreland</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/darebin">Darebin</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/frankston">Frankston</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/melbourne">Melbourne</a></td>,
 <td class="COL1 LGA"><a class="metro" href="/vic/cardinia">Cardinia</a></td>,
 <td class="COL1 LGA"><a class="regional" href="

In [100]:
results = list()
for link in links:
    results.append(link.find("a", href = re.compile("/vic/[a-z]*")))  #extract only a tags with href tags that match the specified pattern and append to list

In [101]:
def openLink(link):
    
    '''
    This function returns the link and the text associated with the link (LGA name).
    '''
    result = link.attrs['href']  
    text = link.text
    return result, text

We'll use the above function to retrieve the link that points to each LGA and use it to extract the relevant table data.

Once the url has been opened, we'll look for all the required table rows based on the tag ('`table`') and its relevant class value ('`DAILY-CASES-BY-LGA`').

We'll extract the table data/text of each row and store its contents in a list named '`content`' and store this in a dictionary named '`t_rows`', with key being the row index (starting from 0) and the value is the content list obtained.

Based on the table headers obtained and stored in the list '`table_headers`', we know what each value in the contents list is supposed to be and so we'll extract only the relevant information, that is the date (index 0) and the cases (index 3) and store them in two lists, '`date`' and '`cases`'. We'll use these lists to create pairs/sublists of the date and corresponding cases and store it in a main list ('`pairs`'). This will be the value to the corresponding key, that is the LGA (obtained from the above function as text) in the '`lga_cases_per_date`' dictionary. The dates will be filtered (based on the month) before they're paired with the cases. The number of cases on 30th September are obtained by subtracting the number of cases in the 0th index and 1st index of the main list (from the dictionary). Similarly, lists that hold the number of cases for the last 14, 30 and 60 days are stored in lists '`last_14_list`', '`last_30_list`' and '`last_60_list`' respectively. These will be used to calculate the rounded average cases (sum of the elements in the list by length of the list, meaning the number of days) in each scenario. The final results are stored in a list, that will be the value to the key (the LGA) in the '`cases_per_lga`' dictionary.

- For the last 14 days, we'll look at cases from 29th September moving back to 16th September (both inclusive).
- For the last 30 days, we'll consider cases from 29th September going back to 31st August (both inclusive).
- For the last 60 days, we'll look at cases from 29th September going back to 1st August (both inclusive).

In [102]:
cases_per_lga = dict()

for idx in range(len(results)):
    result = openLink(results[idx])  #call function to obtain the link and text/phrase
    text = result[1]  #the LGA name
    
    src_5 = urlopen("https://covidlive.com.au/" + result[0])  #open the specified url
    soupObj = BeautifulSoup(src_5, 'html.parser')
    table = soupObj.find("table", {"DAILY-CASES-BY-LGA"}).findAll("tr")
    
    #extraction of table headers
    table_headers = list()

    for th in table[0].findAll("th"):
        table_headers.append(th.text)
    
    
    t_rows = dict()
    
    i = 0
    #extraction of row data
    for ix in range(len(table)):
        content = list()
        for row in table[ix].findAll("td"):
            content.append(row.text)
        t_rows[i] = content
        i += 1
        
    t_rows.pop(0)  #popped the first element since its an empty string
    
    date = []
    cases = []
    #extraction of dates and corresponding cases
    for value in t_rows.values():
        date.append(value[0])
        value[3] = value[3].replace(",", "")  #replacing commas in the cases to later convert into int
        cases.append(int(value[3]))
        
    lga_cases_per_date = dict()
    pairs = list()
    
    for ix in range(len(date)):
        day, month = date[ix].split()  #splitting of date for checking the month
        if month == 'Sep' or month == 'Aug' or date[ix] == "31 Jul":  
            pairs.append([date[ix], cases[ix]])  #date-cases pairs
            
    lga_cases_per_date[text] = pairs  
    
    number_of_cases = dict()  #will store the cases in each scenario for each LGA

    on_30_Sep = lga_cases_per_date[text][0][1] - lga_cases_per_date[text][1][1]  #number of cases added on 30th September

    last_14_list = lga_cases_per_date[text][1:16]  #list of cases in the last 14 days
    last_30_list = lga_cases_per_date[text][1:32]  #list of cases in the last 30 days
    last_60_list = lga_cases_per_date[text][1:]   #list of cases in the last 60 days
    
    
    actual_cases = list()

    for idx in range(len(last_14_list) - 1):
        actual_cases.append(last_14_list[idx][1] - last_14_list[idx + 1][1])
        
    last_14 = float(round(sum(actual_cases) / 14))
    
    actual_cases = list()

    for idx in range(len(last_30_list) - 1):
        actual_cases.append(last_30_list[idx][1] - last_30_list[idx + 1][1])
    
    last_30 = float(round(sum(actual_cases) / 30))
    
    actual_cases = list()

    for idx in range(len(last_60_list) - 1):
        actual_cases.append(last_60_list[idx][1] - last_60_list[idx + 1][1])
        
    last_60 = float(round(sum(actual_cases) / 60))
    
    cases_per_lga[text.upper().strip()] = list()
    cases_per_lga[text.upper().strip()].extend([on_30_Sep, last_14, last_30, last_60])

The dates are in descending order in each list ([29th Sept, 28th Sept, 27th Sept ...]), hence for the number of cases on a particular date, the value is obtained by the subtracting the number of cases on the day you are calculating for and the adjacent date (that comes before it). This gives us the increase in the number of the cases on the present day since the previous day.
- For example, the number of cases on 29th September will be: Cases on 29th - Cases on 28th
- The number of cases on 28th September will be: Cases on 28th - Cases on 27th and so on

The values for '`30_sep_cases`' are calculated on the basis of the same.

In [103]:
table_headers

['DATE', '15+', 'SECOND', 'CASES', 'ACTIVE', 'VAR', 'NET']

In [104]:
cases_per_lga

{'CASEY': [69, 23.0, 14.0, 7.0],
 'GREATER DANDENONG': [36, 10.0, 6.0, 3.0],
 'WYNDHAM': [99, 45.0, 34.0, 20.0],
 'MELTON': [87, 30.0, 19.0, 10.0],
 'HUME': [332, 227.0, 156.0, 80.0],
 'WHITTLESEA': [152, 72.0, 45.0, 23.0],
 'BRIMBANK': [77, 36.0, 21.0, 12.0],
 'MORELAND': [111, 55.0, 48.0, 25.0],
 'DAREBIN': [41, 22.0, 15.0, 8.0],
 'FRANKSTON': [3, 3.0, 2.0, 1.0],
 'MELBOURNE': [23, 12.0, 8.0, 5.0],
 'CARDINIA': [25, 10.0, 5.0, 3.0],
 'LATROBE': [2, 2.0, 1.0, 0.0],
 'GLEN EIRA': [15, 4.0, 2.0, 1.0],
 'MOONEE VALLEY': [41, 18.0, 12.0, 6.0],
 'GREATER GEELONG': [1, 2.0, 2.0, 1.0],
 'KINGSTON': [13, 4.0, 2.0, 1.0],
 'MONASH': [16, 5.0, 3.0, 2.0],
 'MARIBYRNONG': [22, 7.0, 5.0, 3.0],
 'HOBSONS BAY': [20, 15.0, 15.0, 10.0],
 'GREATER SHEPPARTON': [5, 0.0, 2.0, 2.0],
 'WODONGA': [3, 0.0, 0.0, 0.0],
 'KNOX': [23, 5.0, 3.0, 2.0],
 'MORNINGTON PENINSULA': [6, 2.0, 1.0, 1.0],
 'PORT PHILLIP': [25, 9.0, 5.0, 3.0],
 'YARRA': [15, 8.0, 5.0, 3.0],
 'GREATER BENDIGO': [0, 0.0, 0.0, 0.0],
 'BANYULE':

#### 3.5.2 '30_sep_cases', 'last_14_days_cases', 'last_30_days_cases' and 'last_60_days_cases' columns

Using the '`cases_per_lga`' dictionary, the values for each property are added.

To avoid problems while updating the values due to inconsistencies in the names of LGA (for example, COLAC OTWAY and COLAC-OTWAY), regex and its methods are being used to verify if they are the same or not and then update the value.

The pattern '`[A-Z]+\W[A-Z]+`' matches words/LGA names (in uppercase) that have been separated by a non-alphanumeric character ('`\W`' matches characters like whitespaces, hyphens etc).

In [105]:
for idx in list(df.index):
    pattern = re.compile(r"[A-Z]+\W[A-Z]+")  
    row_lga = df['lga'][idx]
    
    #if LGA matches pattern, it is split
    if re.match(pattern, row_lga):  
        result_lga_one = re.split(r"\W", row_lga)
        lga_list = list(cases_per_lga.keys())
        
        #and verified with the LGAs (that match the pattern) present in the dictionary
        for lga in lga_list:
            if re.match(pattern, lga):
                result_lga_two = re.split(r"\W", lga)
                
                if result_lga_one == result_lga_two: #if they're the same, update the values
                    
                    df.at[idx, '30_sep_cases'] = str(cases_per_lga[lga][0])
                    df.at[idx, 'last_14_days_cases'] = str(cases_per_lga[lga][1])
                    df.at[idx, 'last_30_days_cases'] = str(cases_per_lga[lga][2])
                    df.at[idx, 'last_60_days_cases'] = str(cases_per_lga[lga][3])
        
    
    else:
        df.at[idx, '30_sep_cases'] = str(cases_per_lga[df['lga'][idx]][0])
        df.at[idx, 'last_14_days_cases'] = str(cases_per_lga[df['lga'][idx]][1])
        df.at[idx, 'last_30_days_cases'] = str(cases_per_lga[df['lga'][idx]][2])
        df.at[idx, 'last_60_days_cases'] = str(cases_per_lga[df['lga'][idx]][3])

In [106]:
df

,property_id,lat,lng,addr_street,suburb,lga,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag,30_sep_cases,last_14_days_cases,last_30_days_cases,last_60_days_cases
0,91099,-37.956402,145.072540,99 Herald Street,CHELTENHAM,YARRA RANGES,19866,1.947320,40.0,1,5,3.0,2.0,1.0
1,65182,-37.806590,145.317902,18 Kilsyth Avenue,KILSYTH,SURF COAST,19877,2.478957,46.0,1,1,0.0,0.0,0.0
2,17661,-37.698942,144.909829,73 AUGUSTINE TERRACE,GLENROY,WYNDHAM,20031,0.679184,30.0,1,99,45.0,34.0,20.0
3,46534,-37.706180,145.139217,125 Ryans Road,ELTHAM NORTH,ARARAT,19987,1.116812,51.0,1,0,0.0,0.0,0.0
4,87356,-37.939583,145.014479,133 Thomas Street,HAMPTON,WYNDHAM,19949,1.155951,Not available,0,99,45.0,34.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247,60069,-37.810080,145.200907,5 Tourello Street,MITCHAM,PORT PHILLIP,19900,1.181700,31.0,1,25,9.0,5.0,3.0
2248,56038,-37.822793,145.066628,95 Broadway,CAMBERWELL,MELBOURNE,19892,0.371542,18.0,1,23,12.0,8.0,5.0
2249,63480,-37.783340,145.287864,13 Aminga Court,CROYDON,CARDINIA,19878,1.490612,42.0,1,25,10.0,5.0,3.0
2251,80587,-37.989635,145.171660,9 Wannon Court,KEYSBOROUGH,KINGSTON,19889,2.160029,43.0,1,13,4.0,2.0,1.0


## 4. Summary

This assessment measured the understanding of importance of integrating data from various sources effectively. The main outcomes achieved include:

- **Data Integration**: Using various libraries and packages helped in integrating data from several sources to create a dataframe.
- **Issues of data integration**: With the help of various methods available, it was possible to detect any issues before and after integration of the data.

## 5. References

- GeoPandas Project. (2021). *geopandas 0.10.0 documentation.* Retrieved from https://geopandas.org/index.html
- Edmilson Junior. (2016, July 7). *subtract two times in python* [Response to]. Retrieved from https://stackoverflow.com/questions/5259882/subtract-two-times-in-python
- Henrikki Tenkanen. (2018). *Intro to Python GIS: Point in Polygon & Intersect.* Retrieved from https://automating-gis-processes.github.io/CSC18/lessons/L4/point-in-polygon.html